In [1]:
cd ..

/Users/abhi/Documents/GitHub/AQT-KimGroup


In [2]:
from keys import keys
from qiskit import IBMQ, QuantumRegister, QuantumCircuit, quantum_info

%matplotlib inline

In [30]:
import numpy as np

In [3]:
# Setup account
provider = IBMQ.enable_account(keys['ibmq_token'])

In [4]:
IBMQ.save_account(keys['ibmq_token'])

configrc.store_credentials:WARNING:2022-12-06 13:22:54,375: Credentials already present. Set overwrite=True to overwrite.


In [5]:
devices = provider.backends(simulator=True, operational=True)
for x in devices:
    print(x.name(), x.configuration().n_qubits, x.status().pending_jobs)

ibmq_qasm_simulator 32 3
simulator_statevector 32 3
simulator_mps 100 4
simulator_extended_stabilizer 63 4
simulator_stabilizer 5000 4


In [6]:
simulator_backend = provider.backend.ibmq_qasm_simulator

Reproducing Qiskit MLE


The following was part of the old notebook but is now deprecated in qiskit - 
`from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter`

In [7]:
from qiskit_experiments.library import StateTomography, tomography

In [8]:
# No. of qubits
n_qubits = 3

# Initialize the quantum register and circuit
q_reg = QuantumRegister(n_qubits)
ghz_circ = QuantumCircuit(q_reg)

# Add operations to the circuit to prepage a GHZ state
ghz_circ.h(0)
for idx in range(n_qubits-1):
    ghz_circ.cx(idx, idx+1)

ghz_circ.draw()

┌───┐          
q0_0: ┤ H ├──■───────
      └───┘┌─┴─┐     
q0_1: ─────┤ X ├──■──
           └───┘┌─┴─┐
q0_2: ──────────┤ X ├
                └───┘

In [12]:
qst_exp = StateTomography(ghz_circ)
qst_data_1 = qst_exp.run(simulator_backend, seed_simulation=100).block_for_results()

Following the introduction to the experiment framework at `https://qiskit.org/documentation/experiments/apidocs/framework.html#module-qiskit_experiments.framework`

In [41]:
# qst_exp_data = qst_exp.run(simulator_backend)

In [16]:
qst_data_1

ExperimentData(StateTomography, cc3c2454-d349-4fd3-a9a0-fdf143c97839, job_ids=['639020b8d26abd241a25a58b'], metadata=<3 items>)

In [43]:
qst_exp_data.save()

Experiment cannot be saved because no experiment service is available. An experiment service is available, for example, when using an IBM Quantum backend.


In [21]:
job = simulator_backend.retrieve_job('639020b8d26abd241a25a58b')
result = job.result()

In [57]:
result.results[26].header.name.split('_')[1][1:-1].split(', ')

['2', '2', '2']

In [61]:
Nq = 3
Nset = 3**Nq

collateddata = np.zeros((2700, 3), dtype=int)
di = 0
for nset in range(Nset):
    setting = result.results[nset].header.name.split('_')[1][1:-1].split(', ')
    setstr = ''
    for nq in range(Nq):
        setstr += setting[nq]
        rawdata = result.get_counts(nset)
    
    for st in rawdata:
        # Must be careful with indexing due to IBMQ's indexing convention
        outcome = np.array([3*int(st[2-i])+int(setstr[i]) for i in range(3)])
        
        collateddata[di:di+rawdata[st]] = outcome
        di += rawdata[st]

di
assert(di==2700)

AssertionError: 

In [27]:
result.data

<bound method Result.data of Result(backend_name='ibmq_qasm_simulator', backend_version='0.11.0', qobj_id='536c4a81-19a1-45fd-83ac-db8b1050e244', job_id='639020b8d26abd241a25a58b', success=True, results=[ExperimentResult(shots=4000, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 1966, '0x7': 2034}), header=QobjExperimentHeader(clbit_labels=[['c', 0], ['c', 1], ['c', 2]], creg_sizes=[['c', 3]], global_phase=0.0, memory_slots=3, metadata={'clbits': [0, 1, 2], 'm_idx': [0, 0, 0]}, n_qubits=3, name='StateTomography_(0, 0, 0)', qreg_sizes=[['q', 3]], qubit_labels=[['q', 0], ['q', 1], ['q', 2]]), status=DONE, metadata={'active_input_qubits': [0, 1, 2], 'batched_shots_optimization': False, 'device': 'CPU', 'fusion': {'enabled': False}, 'input_qubit_map': [[2, 2], [1, 1], [0, 0]], 'measure_sampling': True, 'method': 'stabilizer', 'noise': 'ideal', 'num_clbits': 3, 'num_qubits': 3, 'parallel_shots': 1, 'parallel_state_update': 16, 'remapped_qubits': False, 'sample_measure_

In [44]:
st_analysis = tomography.StateTomographyAnalysis()
qst_exp_res = st_analysis.run(qst_exp_data)

In [45]:
qst_exp_res.data()

[{'counts': {'000': 2005, '111': 1995},
  'job_id': '636c55240565be2b3600742c',
  'metadata': {'clbits': [0, 1, 2], 'm_idx': [0, 0, 0]},
  'shots': 4000,
  'meas_level': <MeasLevel.CLASSIFIED: 2>},
 {'counts': {'000': 1013, '011': 942, '100': 1042, '111': 1003},
  'job_id': '636c55240565be2b3600742c',
  'metadata': {'clbits': [0, 1, 2], 'm_idx': [0, 0, 1]},
  'shots': 4000,
  'meas_level': <MeasLevel.CLASSIFIED: 2>},
 {'counts': {'000': 1019, '011': 1005, '100': 980, '111': 996},
  'job_id': '636c55240565be2b3600742c',
  'metadata': {'clbits': [0, 1, 2], 'm_idx': [0, 0, 2]},
  'shots': 4000,
  'meas_level': <MeasLevel.CLASSIFIED: 2>},
 {'counts': {'000': 951, '010': 990, '101': 1017, '111': 1042},
  'job_id': '636c55240565be2b3600742c',
  'metadata': {'clbits': [0, 1, 2], 'm_idx': [0, 1, 0]},
  'shots': 4000,
  'meas_level': <MeasLevel.CLASSIFIED: 2>},
 {'counts': {'000': 482,
   '001': 490,
   '010': 517,
   '011': 468,
   '100': 511,
   '101': 517,
   '110': 513,
   '111': 502},
  'j

In [46]:
qst_exp_data = qst_exp_res.data()
keys = [qst_exp_data[i].keys() for i in range(len(qst_exp_data))]
print(keys)

[dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'shots', 'meas_level']), dict_keys(['counts', 'job_id', 'metadata', 'sho

In [49]:
pauli_basis = tomography.basis.PauliMeasurementBasis

rho_li = tomography.fitters.linear_inversion(None, None, None,
                                             measurement_basis=pauli_basis)

TypeError: linear_inversion() missing 3 required positional arguments: 'shot_data', 'measurement_data', and 'preparation_data'

In [51]:
pauli_basis = tomography.basis.PauliMeasurementBasis()
